In [ ]:
import pymysql
import json
import os
import pandas as pd



# Set this to "GCP" or "AWS" (or use an env var: DB_PLATFORM=GCP/AWS)
PLATFORM = os.getenv("DB_PLATFORM", "AWS").upper()

def getconn():
    return pymysql.connect(
    host="database-1.c9ikwq088lg0.us-east-2.rds.amazonaws.com",  # RDS endpoint = host
    port=3306, 
    user="root",                                   # MySQL default port      user="admin",
    password="Casey9203",
    database=None,
    ssl={"ca": "global-bundle.pem"},                    
)


def setup_db(cur):
  # Set up db
    cur.execute('CREATE DATABASE IF NOT EXISTS lifestyle_db')
    cur.execute('USE lifestyle_db')

    
    cur.execute('DROP TABLE IF EXISTS Diet;')
    cur.execute('DROP TABLE IF EXISTS Health')
    cur.execute('DROP TABLE IF EXISTS PersonWorkout;')
    cur.execute('DROP TABLE IF EXISTS Workout;')
    cur.execute('DROP TABLE IF EXISTS Person;')



    cur.execute("""
        CREATE TABLE Person (
            personID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            age INT NOT NULL,
            gender VARCHAR(10),
            weight INT,
            height FLOAT
        );
    """)

    cur.execute("""
        CREATE TABLE Workout (
            workoutID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            session_duration FLOAT,
            calories_burned FLOAT,
            workout_type VARCHAR(30),
            name_of_exercise VARCHAR(50),
            sets FLOAT,
            reps FLOAT,
            body_part VARCHAR(30)
        );
    """)

    cur.execute("""
        CREATE TABLE PersonWorkout (
            personID INT,
            workoutID INT,
            PRIMARY KEY (personID, workoutID),
            FOREIGN KEY (personID) REFERENCES Person(personID),
            FOREIGN KEY (workoutID) REFERENCES Workout(workoutID)
        );
    """)


    cur.execute("""
        CREATE TABLE Diet (
            dietID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            personID INT NOT NULL,
            water_intake FLOAT,
            carbs FLOAT,
            proteins FLOAT,
            fats FLOAT,
            calories FLOAT,
            diet_type VARCHAR(20),
            serving_size FLOAT,
            FOREIGN KEY (personID) REFERENCES Person(personID)
        );
    """)

    cur.execute("""
        CREATE TABLE Health (
            healthID INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            personID INT NOT NULL,
            max_BPM FLOAT,
            resting_BPM FLOAT,
            fat_percentage FLOAT,
            cholesterol FLOAT,
            FOREIGN KEY (personID) REFERENCES Person(personID)
        );
    """)


def insert_data(cur):

    df = pd.read_csv("lifestyle_data.csv")
    df.to_json("lifestyle_data.json", orient="records", indent=2)
    print("converted to json!")
    cur.execute('USE roster_db')

    fname = 'lifestyle_data.json'

    #Data structure as follows:
    #   [
    #   [ "Charley", "si110", 1 ],
    #   [ "Mea", "si110", 0 ],

    # open the file and read 
    str_data = open(fname).read()
    # load the data in a json object
    json_data = json.loads(str_data)

    #json data is loaded in a pyton list
    for entry in json_data:
        print(entry)
        name = entry[0]
        title = entry[1]

        print(name)
        print(title)

        # INSERT OR IGNORE satisfies the uniqueness constraint. the inserted data will be ignored if we try to add duplicates.
        # works as both insert and update
        cur.execute('''INSERT IGNORE INTO User (name)  
            VALUES ( %s )''', (name) )
            
        # look up the primary key from inserted data.		
        cur.execute('SELECT id FROM User WHERE name = %s ', (name, ))
        user_id = cur.fetchone()[0]

        # same technique is used to insert the title
        cur.execute('''INSERT IGNORE INTO Course (title) 
            VALUES ( %s )''', ( title, ) )
        cur.execute('SELECT id FROM Course WHERE title = %s ', (title, ))
        course_id = cur.fetchone()[0]
        
        #insert both keys in the many to many connector table.
        cur.execute('''INSERT IGNORE INTO Member
            (user_id, course_id) VALUES ( %s, %s )''', 
            ( user_id, course_id ) )
    



cnx = getconn()
cur = cnx.cursor()
print("Starting Setup...")
setup_db(cur)
print("Finished Setup.")
print("Starting Insert...")
insert_data(cur)
#print("Finished Insert.")
cur.close()
cnx.commit()
cnx.close()
print("FINISHED")


Starting Setup...
Finished Setup.
Starting Insert...
converted to json!


TypeError: 'NoneType' object is not iterable